In [1]:
 from pororo import Pororo

In [2]:
ner = Pororo(task="ner", lang="ko")


In [3]:
res = ner("손흥민은 28세의 183 센티미터, 77 킬로그램이며, 현재 주급은 약 3억 원이다.", apply_wsd=True)
res[-1][1]

'O'

In [4]:
import json
from konlpy.tag import Mecab
mecab = Mecab()

In [5]:
# Mecab 사용해보기
print(mecab.morphs(u'우리나라에는 무릎 치료를 잘하는 정형외과가 없는가!'))
print(mecab.pos(u'우리나라에는 무릎 치료를 잘하는 정형외과가 없는가!'))

['우리', '나라', '에', '는', '무릎', '치료', '를', '잘', '하', '는', '정형', '외과', '가', '없', '는가', '!']
[('우리', 'NP'), ('나라', 'NNG'), ('에', 'JKB'), ('는', 'JX'), ('무릎', 'NNG'), ('치료', 'NNG'), ('를', 'JKO'), ('잘', 'MAG'), ('하', 'VV'), ('는', 'ETM'), ('정형', 'NNG'), ('외과', 'NNG'), ('가', 'JKS'), ('없', 'VA'), ('는가', 'EF'), ('!', 'SF')]


In [6]:
type(mecab.pos(u'우리나라에는 무릎 치료를 잘하는 정형외과가 없는가!')[0])

tuple

In [7]:
MORPH = 0
POS = 1  # part of speech

def tag_and_remove_particle(text):
    res = []
    spaced = text.split(' ')
    for word in spaced:
        res.extend(mecab.pos(word))
        res.append(' ')

    end = len(res) - 1
    for i in range(len(res)-1, -1, -1):
        if res[i] == ' ':
            continue
        if res[i][POS][0] != 'J':
            end = i
            break
    
    # return ' '.join(mecab.morphs(text)[:end+1])
    morphs = [t[MORPH] if isinstance(t, tuple) else ' ' for t in res[:end+1]]
    return ''.join(morphs).rstrip()

In [8]:
# text = "지질 과산화 생성물이"
text = '준융합성 천연두'
print(f'Raw NER: {ner(text, apply_wsd=True)}\n')
print(f'Raw Morphs: {mecab.morphs(text)}\n')

processed = tag_and_remove_particle(text)
print(f'[{processed}]')
print(f'Morphs aware: {ner(processed, apply_wsd=True)}')

Raw NER: [('준융합성', 'O'), (' ', 'O'), ('천연두', 'DISEASE')]

Raw Morphs: ['준', '융합', '성', '천연', '두']

[준융합성 천연]
Morphs aware: [('준융합성', 'O'), (' ', 'O'), ('천연', 'O')]


In [9]:
target_json = '/opt/ml/output/predictions.json'
with open(target_json) as json_data:
    data = json.load(json_data)


In [10]:
import os
from collections import OrderedDict

processed_json = OrderedDict()
fixed = []

for k, v in data.items():
    v_fixed = tag_and_remove_particle(v)
    if v_fixed != v:
        print(f'{k}: Fixed {v} -> {v_fixed}')
        processed_json[k] = v_fixed
        fixed.append((v,v_fixed))
    else:
        processed_json[k] = v

print(f'Fixed total {len(fixed)} entries.')
with open(target_json.split('.')[0]+'_fixed.json', "w") as fp:
    json.dump(processed_json, fp, ensure_ascii=False, indent = 4)
    

mrc-0-004007: Fixed 괴팅겐 -> 괴팅
mrc-0-005278: Fixed 수오멘 소시알리데모크라티넨 푸올루에 -> 수오멘 소시알리데모크라티넨 푸올루
mrc-0-005283: Fixed 감각기에 -> 감각기
mrc-0-001149: Fixed 인공어

인공어 -> 인공어인공어
mrc-1-001592: Fixed 일본국유철도(이하 국철)과 JR의 노선이 있는 구간이 많다.

국철이 JR -> 일본국유철도(이하 국철)과 JR의 노선이 있는 구간이 많다.국철이 JR
mrc-0-002758: Fixed 우크라이나 -> 우크라
mrc-0-001076: Fixed 베다는 -> 베다
mrc-0-004759: Fixed 가압 카트리지가 -> 가압 카트리지
mrc-0-002289: Fixed 가덕도 -> 가덕
mrc-0-002245: Fixed 던전

던전 -> 던전던전
mrc-1-000841: Fixed 조선총독부에 빌붙고 일제의 통치를 찬양하는 일부 구 대한제국 대신들의 -> 조선총독부에 빌붙고 일제의 통치를 찬양하는 일부 구 대한제국 대신들
Fixed total 11 entries.


In [11]:
processed_json = OrderedDict()
fixed_v2 = []

# V2 only attempts to fix output if last element is not 'O'
# '천연두' -> '천연' + '두' 로 잘리는 것 방지
# Issue? (정답인지 아닌지 모르지만) 원래 정답에 있던 \n 가 fix 과정에서 사라지는 문제
for k, v in data.items():
    if ner(v)[-1][POS] != 'O':
        processed_json[k] = v
    else:
        v_fixed = tag_and_remove_particle(v)
        if v_fixed != v:
            print(f'{k}: Fixed {v} -> {v_fixed}')
            processed_json[k] = v_fixed
            fixed_v2.append((v,v_fixed))
        else:
            processed_json[k] = v

print(f'Fixed total {len(fixed_v2)} entries.')
with open(target_json.split('.')[0]+'_fixed_v2.json', "w") as fp:
    json.dump(processed_json, fp, ensure_ascii=False, indent = 4)

mrc-0-005283: Fixed 감각기에 -> 감각기
mrc-0-001149: Fixed 인공어

인공어 -> 인공어인공어
mrc-0-001076: Fixed 베다는 -> 베다
mrc-0-004759: Fixed 가압 카트리지가 -> 가압 카트리지
mrc-0-002245: Fixed 던전

던전 -> 던전던전
mrc-1-000841: Fixed 조선총독부에 빌붙고 일제의 통치를 찬양하는 일부 구 대한제국 대신들의 -> 조선총독부에 빌붙고 일제의 통치를 찬양하는 일부 구 대한제국 대신들
Fixed total 6 entries.


In [12]:
diff = set(fixed) - set(fixed_v2)
print(diff)

{('수오멘 소시알리데모크라티넨 푸올루에', '수오멘 소시알리데모크라티넨 푸올루'), ('가덕도', '가덕'), ('우크라이나', '우크라'), ('괴팅겐', '괴팅'), ('일본국유철도(이하 국철)과 JR의 노선이 있는 구간이 많다.\n\n국철이 JR', '일본국유철도(이하 국철)과 JR의 노선이 있는 구간이 많다.국철이 JR')}
